<a href="https://colab.research.google.com/github/harshkodan/Aetherbot/blob/main/midsemmajorproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q "requests>=2.32.4,<2.33" \
  langchain langchain-community langchain-core \
  langchain-google-genai sentence-transformers \
  chromadb gradio google-generativeai python-docx reportlab


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 100.9 MB/s eta 0:00

In [ ]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

BASE = Path("/content/drive/MyDrive/aether_data")
BASE.mkdir(exist_ok=True)
print(" Drive mounted and data directory ready:", BASE)


Mounted at /content/drive
 Drive mounted and data directory ready: /content/drive/MyDrive/aether_data


In [ ]:
import os, json
from pathlib import Path
from getpass import getpass

CONFIG_PATH = BASE / "config.json"

def save_key_interactive():
    key = getpass("Paste your Google Gemini API key (hidden): ").strip()
    if key:
        CONFIG_PATH.write_text(json.dumps({"GEMINI_KEY": key}))
        os.environ["GOOGLE_API_KEY"] = key
        print(" Key saved securely to Drive.")
    else:
        print(" No key entered.")

def load_key():
    if CONFIG_PATH.exists():
        try:
            data = json.loads(CONFIG_PATH.read_text())
            key = data.get("GEMINI_KEY")
            if key:
                os.environ["GOOGLE_API_KEY"] = key
                print(" Gemini key loaded from Drive.")
                return True
        except Exception as e:
            print(" Could not read config:", e)
    print(" No config found. Run save_key_interactive() to store your key.")
    return False

load_key()


 Gemini key loaded from Drive.


True

In [ ]:
import textwrap

files_data = {
    "university_info": """Title: University Overview - UPES
UPES (University of Petroleum and Energy Studies) is a private university located in Dehradun, India.
Campus: Energy Acres, Nehru Colony. Main administrative block, library, and placement cell are located on Energy Acres campus.
Contact: admissions@upes.ac.in | Phone: 0135-2770137
Key facts:
- Established in 2003.
- Major schools: Computer Science, Energy, Business, Law, Design.
- Notable facilities: Central library, sports complex, on-campus hospital, gym.
""",

    "admissions": """Title: Admissions - Overview
Admissions to various programs (B.Tech, M.Tech, MBA, MSc) are coordinated by the Admissions Office.
Typical schedule:
- B.Tech: applications open April; counseling and seat allocation April–June.
- MBA: applications open December–February; final rounds March–April.
Eligibility and process:
- B.Tech: 10+2 with Physics, Maths, English; minimum aggregate 50% or based on entrance test.
- MBA: Graduation with minimum 50% or entrance score (CAT/MAT/UPES).
Documents required: application form, 10th/12th marksheet, graduation marksheet (if any), ID proof, passport-size photo.
""",

    "placements": """Title: Placements & Internships
Placement cell: Located at Energy Acres campus (Placement Cell Office, Block C).
Typical timeline: Pre-placement talks Jan–Feb; main drives March–May.
Recruiters (examples): Amazon, Microsoft, Google, TCS, Infosys, Cognizant, Accenture.
Placement stats (example batch):
- Overall placement rate: 87%
- Highest package: INR 45 LPA
- Average package (CSE): INR 12.5 LPA
Internships:
- Internship policy: students are strongly encouraged to do internships in 3rd year; some programs require mandatory summer internships.
Contact: placements@upes.ac.in
""",

    "courses": """Title: Programs & Courses (AIML, CSE, MBA etc.)
Example: B.Tech AIML — 4 years
- Year 1: Basic programming, calculus, physics, engineering drawing.
- Year 2: Data Structures, Probability & Statistics, Linear Algebra.
- Year 3: Machine Learning, Computer Vision, NLP, Electives.
- Year 4: Capstone project, advanced electives, internship wrap-up.
Assessment: combination of midterm, assignments, practical labs, final exam, and project evaluations.
""",

    "hostel_facilities": """Title: Hostel & Student Accommodation
Hostel office: Located near Energy Acres Administration building.
Facilities:
- Room types: single, double, triple occupancy.
- Mess: vegetarian and non-vegetarian options, monthly mess menu posted online.
- Amenities: Wi-Fi, laundry services, on-floor common room, CCTV in public areas.
Rules:
- Minimum attendance requirement and regular ID card checks.
- Guests allowed with prior permission; curfew times enforced for security.
Fees: Updated annually — contact hostel office for current fee structure.
""",

    "faculty": """Title: Faculty & Administration
Dean of School of Computer Science: Prof. A. Example (contact via department email).
Faculty roles:
- Head of Department (HoD) coordinates curriculum, exam schedules, faculty allocation.
- Research groups: AI Lab, Energy Systems Lab, Data Analytics Lab.
Office hours: Typically 10:00–16:00; students may book slots through departmental portal.
""",

    "scholarships": """Title: Scholarships & Financial Aid
Scholarship categories:
- Merit-based scholarships: awarded to top-rank students based on entrance scores and academic performance.
- Need-based scholarships: require submission of family income documents and supporting statements.
Application process:
- Scholarship applications open in the second semester for current students and during admission for freshers.
- Contact: scholarships@upes.ac.in
""",

    "student_life": """Title: Student Life, Clubs & Events
Clubs and societies: Coding Club, IEEE Student Chapter, Literary Society, Adventure Club.
Annual events: Uurja (tech fest), cultural week, sports meet (inter-college).
Facilities: Canteen, student health center, counseling services, placement workshops.
Student support: career counseling, mentoring by seniors, alumni mentorship programs.
"""
}

for filename, data in files_data.items():
    path = BASE / filename
    if not path.exists():
        path.write_text(textwrap.dedent(data))
        print(f" Created {filename}")
    else:
        print(f" Exists: {filename}")

print(" Dataset ready with", len(files_data), "text files.")


 Created university_info
 Created admissions
 Created placements
 Created courses
 Created hostel_facilities
 Created faculty
 Created scholarships
 Created student_life
 Dataset ready with 8 text files.


In [ ]:
from pathlib import Path
import zipfile

EXPANDED = BASE / "expanded_corpus"
EXPANDED.mkdir(parents=True, exist_ok=True)

zip_path = BASE / "expanded_corpus.zip"

if zip_path.exists():
    print(" Found expanded_corpus.zip — unzipping...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(EXPANDED)
    print(" Unzipped to", EXPANDED)
else:
    print(" No expanded_corpus.zip found. You can manually upload .txt/.pdf/.docx files to:", EXPANDED)


 No expanded_corpus.zip found. You can manually upload .txt/.pdf/.docx files to: /content/drive/MyDrive/aether_data/expanded_corpus


In [ ]:

import yaml
from pathlib import Path
import textwrap


BASE = Path("/content/drive/MyDrive/aether_data")
UPLOADED_DIR = BASE
EXPANDED = BASE / "expanded_corpus"
EXPANDED.mkdir(parents=True, exist_ok=True)

candidates = {
    "nlu.yml": UPLOADED_DIR / "nlu.yml",
    "stories.yml": UPLOADED_DIR / "stories.yml",
    "rules.yml": UPLOADED_DIR / "rules.yml"
}


In [ ]:

import yaml
from pathlib import Path
import textwrap

BASE = Path("/content/drive/MyDrive/aether_data")
UPLOADED_DIR = BASE
EXPANDED = BASE / "expanded_corpus"
EXPANDED.mkdir(parents=True, exist_ok=True)

candidates = {
    "nlu.yml": UPLOADED_DIR / "nlu.yml",
    "stories.yml": UPLOADED_DIR / "stories.yml",
    "rules.yml": UPLOADED_DIR / "rules.yml"
}

created = []
for name, path in candidates.items():
    if not path.exists():
        print(f" {name} not found at {path} — skipping.")
        continue
    try:
        data = yaml.safe_load(path.read_text())
    except Exception as e:
        print(f" Failed to parse {name}: {e}")
        continue

    if name == "nlu.yml":
        lines = ["Title: NLU training examples (converted from nlu.yml)", "-"*60, ""]
        items = data.get("nlu") if isinstance(data, dict) else data
        if not items:
            items = data if isinstance(data, list) else []
        for block in items or []:
            intent = block.get("intent") or block.get("name") or "<unknown-intent>"
            examples = block.get("examples") or block.get("text") or ""
            if isinstance(examples, str):
                ex_list = [line.strip().lstrip("-").strip() for line in examples.splitlines() if line.strip()]
            elif isinstance(examples, list):
                ex_list = [str(x).strip() for x in examples]
            else:
                ex_list = []
            if ex_list:
                lines.append(f"Intent: {intent}")
                for ex in ex_list:
                    lines.append(" - " + ex)
                lines.append("")
        out_text = "\n".join(lines).strip()
        out_path = EXPANDED / "nlu_examples.txt"
        out_path.write_text(textwrap.dedent(out_text))
        created.append(out_path)

    elif name == "stories.yml":
        lines = ["Title: Stories / conversation flows (converted from stories.yml)", "-"*60, ""]
        items = data.get("stories") if isinstance(data, dict) else data
        for s in items or []:
            name_s = s.get("story") or s.get("name") or "<unnamed story>"
            lines.append(f"Story: {name_s}")
            steps = s.get("steps") or s.get("steps", [])
            if isinstance(steps, list):
                for st in steps:
                    if isinstance(st, dict):
                        if "intent" in st:
                            lines.append("  - Intent: " + (st.get("intent") or ""))
                        if "action" in st:
                            lines.append("  - Action: " + (st.get("action") or ""))
                        if "user" in st:
                            lines.append("  - User: " + str(st.get("user")))
                    else:
                        lines.append("  - " + str(st))
            else:
                lines.append("  " + str(steps))
            lines.append("")
        out_path = EXPANDED / "stories_summary.txt"
        out_path.write_text(textwrap.dedent("\n".join(lines)))
        created.append(out_path)

    elif name == "rules.yml":
        lines = ["Title: Rules (converted from rules.yml)", "-"*60, ""]
        items = data.get("rules") if isinstance(data, dict) else data
        for r in items or []:
            rule_name = r.get("rule") or r.get("name") or "<unnamed rule>"
            lines.append(f"Rule: {rule_name}")
            steps = r.get("steps") or r.get("condition") or r.get("action") or []
            if isinstance(steps, list):
                for st in steps:
                    if isinstance(st, dict):
                        if "intent" in st:
                            lines.append("  - Intent: " + st.get("intent", ""))
                        if "action" in st:
                            lines.append("  - Action: " + st.get("action", ""))
                    else:
                        lines.append("  - " + str(st))
            else:
                lines.append("  " + str(steps))
            lines.append("")
        out_path = EXPANDED / "rules_summary.txt"
        out_path.write_text(textwrap.dedent("\n".join(lines)))
        created.append(out_path)

print("\n Conversion complete. Created files:")
for p in created:
    print("-", p.name, f"({p.stat().st_size} bytes)")
    txt = p.read_text()
    preview = txt[:400].replace("\n"," ")
    print("  preview:", preview[:300] + ("..." if len(preview)>300 else ""))
print("\nNext step: run your incremental ingestion cell (the one that indexes only new files).")



 Conversion complete. Created files:
- nlu_examples.txt (2107 bytes)
  preview: Title: NLU training examples (converted from nlu.yml) ------------------------------------------------------------  Intent: greet  - hey  - hello  - hi  - hello there  - good morning  - good evening  - moin  - hey there  - let's go  - hey dude  - goodmorning  - goodevening  - good afternoon  Intent:...
- stories_summary.txt (1613 bytes)
  preview: Title: Stories / conversation flows (converted from stories.yml) ------------------------------------------------------------  Story: happy path   - Intent: greet   - Action: utter_greet   - Intent: mood_great   - Action: utter_happy  Story: sad path 1   - Intent: greet   - Action: utter_greet   - I...
- rules_summary.txt (396 bytes)
  preview: Title: Rules (converted from rules.yml) ------------------------------------------------------------  Rule: Say goodbye anytime the user says goodbye   - Intent: out_of_scope   - Action: action_out_of_scope  Rule: Out of s

In [ ]:
from pathlib import Path
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

DB_DIR = Path("/content/drive/MyDrive/aether_data/chroma_db")
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma(persist_directory=str(DB_DIR), embedding_function=embedding)
try:
    count = vectordb._collection.count()
except Exception:
    docs = vectordb.similarity_search("test", k=1)
    count = len(docs)
print("Estimated DB chunks:", count)


/tmp/ipython-input-4134771526.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-4134771526.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=str(DB_DIR), embedding_function=embedding)


Estimated DB chunks: 406


In [ ]:
from pathlib import Path
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, PyPDFLoader, Docx2txtLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

DB_DIR = BASE / "chroma_db"
PROCESSED = BASE / "processed_files.json"

def read_processed():
    if PROCESSED.exists():
        try:
            return set(json.loads(PROCESSED.read_text()).get("files", []))
        except:
            return set()
    return set()

def write_processed(files_set):
    PROCESSED.write_text(json.dumps({"files": sorted(list(files_set))}, indent=2))

candidates = []
for p in BASE.rglob("*"):
    if p.is_file() and p.suffix.lower() in [".txt", ".pdf", ".docx"]:
        candidates.append(p)

if EXPANDED.exists():
    for p in EXPANDED.rglob("*"):
        if p.is_file() and p.suffix.lower() in [".txt", ".pdf", ".docx"]:
            if p not in candidates:
                candidates.append(p)

processed = read_processed()
to_index = [p for p in sorted(set(candidates), key=lambda x: str(x)) if str(p) not in processed]

print("New files to index:", [p.name for p in to_index])

if not to_index:
    print(" No new files to ingest.")
else:
    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
    all_docs = []
    for p in to_index:
        try:
            if p.suffix.lower() == ".txt":
                loader = TextLoader(str(p))
            elif p.suffix.lower() == ".pdf":
                loader = PyPDFLoader(str(p))
            else:
                loader = Docx2txtLoader(str(p))
            raw = loader.load()
            for d in raw:
                d.metadata = d.metadata or {}
                d.metadata["source"] = str(p)
            chunks = splitter.split_documents(raw)
            all_docs.extend(chunks)
            print(f" {p.name} -> {len(chunks)} chunks")
        except Exception as e:
            print(" Error loading", p.name, e)

    if all_docs:
        emb = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
        vectordb = Chroma.from_documents(all_docs, emb, persist_directory=str(DB_DIR))
        vectordb.persist()
        processed.update([str(p) for p in to_index])
        write_processed(processed)
        print(" Indexed new files successfully.")


New files to index: ['Syllabus_Highlights.pdf', 'admissions.txt', 'courses.txt', 'faculty.txt', 'hostel_facilities.txt', 'nlu_examples.txt', 'placement_guidelines_expanded.docx', 'placements.txt', 'rules_summary.txt', 'scholarships.txt', 'stories_summary.txt', 'student_life.txt', 'university_info.txt', 'faq.txt']
 Error loading Syllabus_Highlights.pdf `pypdf` package not found, please install it with `pip install pypdf`
 admissions.txt -> 7 chunks
 courses.txt -> 9 chunks
 faculty.txt -> 5 chunks
 hostel_facilities.txt -> 5 chunks
 nlu_examples.txt -> 3 chunks
 Error loading placement_guidelines_expanded.docx No module named 'docx2txt'
 placements.txt -> 6 chunks
 rules_summary.txt -> 1 chunks
 scholarships.txt -> 4 chunks
 stories_summary.txt -> 3 chunks
 student_life.txt -> 5 chunks
 university_info.txt -> 2 chunks
 faq.txt -> 1 chunks
 Indexed new files successfully.


/tmp/ipython-input-883349968.py:64: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
from pathlib import Path
import json, shutil
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, PyPDFLoader, Docx2txtLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

DB_DIR = BASE / "chroma_db"
PROCESSED = BASE / "processed_files.json"

if PROCESSED.exists():
    PROCESSED.unlink()
    print(" Removed processed index.")

if DB_DIR.exists():
    pass

print("Rebuilding Chroma DB...")

files = [p for p in BASE.rglob("*") if p.is_file() and p.suffix.lower() in [".txt", ".pdf", ".docx"]]

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
all_docs = []
for f in files:
    try:
        if f.suffix.lower() == ".txt":
            loader = TextLoader(str(f))
        elif f.suffix.lower() == ".pdf":
            loader = PyPDFLoader(str(f))
        else:
            loader = Docx2txtLoader(str(f))
        raw = loader.load()
        for d in raw:
            d.metadata = d.metadata or {}
            d.metadata["source"] = str(f)
        chunks = splitter.split_documents(raw)
        all_docs.extend(chunks)
        print(f" {f.name} -> {len(chunks)} chunks")
    except Exception as e:
        print(" Error loading", f.name, e)

embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(all_docs, embedding, persist_directory=str(DB_DIR))
vectordb.persist()
print(" Full rebuild complete with", len(all_docs), "chunks.")


Rebuilding Chroma DB...
 faq.txt -> 1 chunks
 rules_summary.txt -> 1 chunks
 university_info.txt -> 2 chunks
 Error loading Syllabus_Highlights.pdf `pypdf` package not found, please install it with `pip install pypdf`
 student_life.txt -> 5 chunks
 stories_summary.txt -> 3 chunks
 scholarships.txt -> 4 chunks
 placements.txt -> 6 chunks
 Error loading placement_guidelines_expanded.docx No module named 'docx2txt'
 nlu_examples.txt -> 3 chunks
 hostel_facilities.txt -> 5 chunks
 faculty.txt -> 5 chunks
 courses.txt -> 9 chunks
 admissions.txt -> 7 chunks
 Full rebuild complete with 51 chunks.


In [ ]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    print(" Key not loaded — run save_key_interactive() or load_key() first.")
else:
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
    MODEL = "models/gemini-2.5-flash"
    print(" Connected to Gemini:", MODEL)


 Connected to Gemini: models/gemini-2.5-flash


In [ ]:
from langchain_community.vectorstores import Chroma
from pathlib import Path

def hybrid_answer(query, k=3):
    embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    retriever = Chroma(persist_directory=str(DB_DIR), embedding_function=embedding).as_retriever(search_kwargs={"k": k})
    docs = retriever.get_relevant_documents(query)

    if not docs:
        chat = ChatGoogleGenerativeAI(model=MODEL, temperature=0.35)
        g = chat.invoke(f"Answer clearly and briefly: {query}")
        return f"{g.content}\n\n(Note: answer generated without local data.)"

    context = "\n\n".join([d.page_content for d in docs])
    prompt = f"Use the following context to answer the question clearly.\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"
    chat = ChatGoogleGenerativeAI(model=MODEL, temperature=0.25)
    response = chat.invoke(prompt)
    sources = ', '.join({Path(d.metadata['source']).name for d in docs[:k]})
    return f"{response.content}\n\n Sources: {sources}"


In [ ]:
import gradio as gr

def smart_aether_response(query, k):
    try:
        return hybrid_answer(query, int(k))
    except Exception as e:
        return f" Error: {e}"

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🎓 AETHER BOT v4 — Smart University Assistant")
    gr.Markdown("Ask anything about UPES admissions, placements, or campus life.")

    q = gr.Textbox(label="Ask Aether Bot", placeholder="e.g. When do admissions start?", lines=2)
    k_slider = gr.Slider(1, 5, value=3, step=1, label="Top-k passages")
    btn = gr.Button("🔍 Get Answer")
    out = gr.Textbox(label="Answer + Sources", lines=20)

    btn.click(fn=smart_aether_response, inputs=[q, k_slider], outputs=out)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://60e8d3a72f55b42013.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, json
from pathlib import Path
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

BASE = Path("/content/drive/MyDrive/aether_data")
DB_DIR = BASE / "chroma_db"
CONFIG_PATH = BASE / "config.json"

if CONFIG_PATH.exists():
    data = json.loads(CONFIG_PATH.read_text())
    os.environ["GOOGLE_API_KEY"] = data.get("GEMINI_KEY", "")
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
    print(" Gemini key loaded.")
else:
    print(" No config.json found. Run the setup cell once.")

embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma(persist_directory=str(DB_DIR), embedding_function=embedding)
print(" Vector DB ready. Estimated chunks:", vectordb._collection.count())

chat = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", temperature=0.2)
test = chat.invoke("Hello, Aether Bot is ready?")
print(" Gemini says:", test.content)

print("\n Aether Bot ready to launch! Run the Gradio cell next.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔑 Gemini key loaded.


/tmp/ipython-input-710866742.py:27: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-710866742.py:28: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=str(DB_DIR), embedding_function=embedding)


📚 Vector DB ready. Estimated chunks: 231
🤖 Gemini says: Hello! I am a large language model, an AI, and I'm ready to assist you.

I don't have a specific name like "Aether Bot," but you can just call me AI or assistant.

How can I help you today?

✅ Aether Bot ready to launch! Run the Gradio cell next.
